In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np

In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDQ87P/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDQ87P/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteDDQ87P/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [5]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [6]:
model = keras.Sequential([
  hub_layer,
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
               loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [8]:
model.fit(train_data.shuffle(10000).batch(512),
           epochs=20,
          validation_data=validation_data.batch(512),
           verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 91ms/step - loss: 1.8226 - accuracy: 0.5007 - val_loss: 0.9106 - val_accuracy: 0.4900
Epoch 2/20
30/30 [==============================] - 3s 87ms/step - loss: 0.8472 - accuracy: 0.4752 - val_loss: 0.7985 - val_accuracy: 0.5057
Epoch 3/20
30/30 [==============================] - 3s 86ms/step - loss: 0.7498 - accuracy: 0.5383 - val_loss: 0.7282 - val_accuracy: 0.5448
Epoch 4/20
30/30 [==============================] - 3s 89ms/step - loss: 0.6824 - accuracy: 0.5785 - val_loss: 0.6680 - val_accuracy: 0.5943
Epoch 5/20
30/30 [==============================] - 3s 90ms/step - loss: 0.6280 - accuracy: 0.6271 - val_loss: 0.6176 - val_accuracy: 0.6304
Epoch 6/20
30/30 [==============================] - 3s 85ms/step - loss: 0.5777 - accuracy: 0.6745 - val_loss: 0.5718 - val_accuracy: 0.6731
Epoch 7/20
30/30 [==============================] - 3s 87ms/step - loss: 0.5303 - accuracy: 0.7193 - val_loss: 0.5285 - val_accuracy: 0.7226
Epoch 8/20
30

In [9]:
test_loss, test_acc = model.evaluate(test_data.batch(512), verbose=2)

print('\nTest accuracy:', test_acc)

49/49 - 3s - loss: 0.3203 - accuracy: 0.8612

Test accuracy: 0.8611599802970886


In [10]:
test_examples_batch, test_labels_batch = next(iter(test_data.batch(10)))
test_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
   

In [11]:
test_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1])>

In [12]:
pred = model.predict(test_examples_batch)
keras.activations.sigmoid(pred)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.4683309 ],
       [0.73800194],
       [0.482514  ],
       [0.14987037],
       [0.9329839 ],
       [0.9955161 ],
       [0.99884284],
       [0.9904963 ],
       [0.9315303 ],
       [0.68585616]], dtype=float32)>